In [ ]:
# (C) Copyright Aaron Goldberg, 2022.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

In [1]:
# We want to know what the PNRDs say when we input 4 equal TMSVs
import numpy as np
import strawberryfields as sf
sf.ping() # Check connection to platform; API key was previously established
from strawberryfields import ops
from strawberryfields import RemoteEngine

prog = sf.Program(8, name="TMSV_stats")

with prog.context as q:
    # Initial squeezed states in pairs of modes
    # Allowed values are r=1.0 or r=0.0; always use r=1.0
    ops.S2gate(1.0) | (q[0], q[4])
    ops.S2gate(1.0) | (q[1], q[5])
    ops.S2gate(1.0) | (q[3], q[7])
    ops.S2gate(1.0) | (q[2], q[6])



    ops.MeasureFock() | q

You have successfully authenticated to the platform!


In [ ]:
eng = RemoteEngine("X8") # This is on the real computer
num_shots=1000000 # Number of repetitions per trial run
num_jobs=1000 # Number of trials
jobs = [eng.run_async(prog, shots=num_shots) for i in range(num_jobs)] # Run asynchronously

In [ ]:
def write_data_to_file(newdata, file_counter):
    # Find probability distributions for pairs of detectors
    maxval=np.amax(newdata) # There's a 9 in there!!;
    offset=4 # Distance between pairs of detectors that share a TMSV
    unique04, counts04 = np.unique(newdata[::,0:5:4], return_counts=True,axis=0)
    unique15, counts15 = np.unique(newdata[::,0+1:5+1:4], return_counts=True,axis=0)
    unique26, counts26 = np.unique(newdata[::,0+2:5+2:4], return_counts=True,axis=0)
    unique37, counts37 = np.unique(newdata[::,0+3:5+3:4], return_counts=True,axis=0)
    
    arranged_counts04=np.zeros((maxval+1,maxval+1))
    for i in range(len(counts04)):
        arranged_counts04[unique04[i,0],unique04[i,1]]=counts04[i]
    arranged_counts15=np.zeros((maxval+1,maxval+1))
    for i in range(len(counts15)):
        arranged_counts15[unique15[i,0],unique15[i,1]]=counts15[i]
    arranged_counts26=np.zeros((maxval+1,maxval+1))
    for i in range(len(counts26)):
        arranged_counts26[unique26[i,0],unique26[i,1]]=counts26[i]
    arranged_counts37=np.zeros((maxval+1,maxval+1))
    for i in range(len(counts37)):
        arranged_counts37[unique37[i,0],unique37[i,1]]=counts37[i]
    # Each pair of detectors and each trial get their own saved histogram
    np.savetxt(f'TMSV_data/04_{file_counter}.csv', arranged_counts04.astype(int), delimiter=',',fmt='%i')
    np.savetxt(f'TMSV_data/15_{file_counter}.csv', arranged_counts15.astype(int), delimiter=',',fmt='%i')
    np.savetxt(f'TMSV_data/26_{file_counter}.csv', arranged_counts26.astype(int), delimiter=',',fmt='%i')
    np.savetxt(f'TMSV_data/37_{file_counter}.csv', arranged_counts37.astype(int), delimiter=',',fmt='%i')

In [ ]:
import time
sleep_time=10 # How many seconds we should wait before checking again whether our job completed
job_counter=0
file_counter=250
while job_counter < num_jobs:
    # This assumes the jobs will execute in order, which should be true at least approximately, for identical jobs. Should not slow things down otherwise
    jobs[job_counter].refresh()
    if jobs[job_counter].status == 'failed':
        print(f'failed at job #{job_counter}')
        job_counter=job_counter+1 # Skip any jobs that failed
    elif jobs[job_counter].status == 'complete':
        if jobs[job_counter].result is None:
            print(f'empty result at job #{job_counter}')
            job_counter=job_counter+1 # Skip any jobs that failed
        else:
            # Write data to file
            write_data_to_file(jobs[job_counter].result.samples, file_counter)
            print(f'complete at job #{job_counter} to make file #{file_counter}')
            jobs[job_counter]=0 # Free up space
            job_counter=job_counter+1
            file_counter=file_counter+1
    else:
        time.sleep(sleep_time) # No need to keep checking too often if the job hasn't completed yet. If a job does complete, we check the next job without waiting